In [1]:
# Import the dependencies.
import pandas as pd
import gmaps
import requests
# Import the API key.
from config import g_key

In [3]:
# Store the CSV you saved created in part one into a DataFrame.
city_data_df = pd.read_csv("weather_data/cities.csv")
city_data_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
0,0,Okhotsk,RU,2022-03-27 17:50:42,59.3833,143.3000,-2.49,85,34,11.97
1,1,Cape Town,ZA,2022-03-27 17:50:42,-33.9258,18.4232,73.81,40,0,4.61
2,2,Busselton,AU,2022-03-27 17:50:43,-33.6500,115.3333,58.21,70,0,14.56
3,3,Pinawa,CA,2022-03-27 17:50:43,50.1489,-95.8811,15.35,53,20,6.26
4,4,East London,ZA,2022-03-27 17:50:43,-33.0153,27.9116,70.74,83,0,11.50


In [4]:
# Configure gmaps to use your Google API key.
gmaps.configure(api_key=g_key)

In [5]:
# Heatmap of temperature
# Get the latitude and longitude.
locations = city_data_df[["Lat", "Lng"]]
# Get the maximum temperature.
max_temp = city_data_df["Max Temp"]

# Get the maximum temperature.
#max_temp = city_data_df["Max Temp"]
#temps = []
#for temp in max_temp:
#    temps.append(max(temp, 0))
    
# Assign the figure variable.
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)

# Assign the heatmap variable.
heat_layer = gmaps.heatmap_layer(locations, weights=[max(temp, 0) for temp in max_temp], dissipating=False, max_intensity=300, point_radius=4)
# Add the heatmap layer.
fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [6]:
# Heatmap of percent humidity
locations = city_data_df[["Lat", "Lng"]]
humidity = city_data_df["Humidity"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [7]:
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 20
What is the maximum temperature you would like for your trip? 25


In [8]:
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
50,50,Kyakhta,RU,2022-03-27 17:51:58,50.3508,106.4494,20.95,87,97,9.10
54,54,Nome,US,2022-03-27 17:51:59,64.5011,-165.4064,20.16,79,100,4.61
62,62,Jamestown,US,2022-03-27 17:49:04,42.0970,-79.2353,24.53,64,100,16.11
180,180,Korsakovo,RU,2022-03-27 17:55:47,53.2677,37.3569,22.55,71,99,18.95
223,223,Sovetskiy,RU,2022-03-27 17:56:02,61.3614,63.5842,23.70,90,100,14.61
246,246,Zabaykalsk,RU,2022-03-27 17:57:10,49.6333,117.3167,21.31,48,21,5.53
290,290,Krasnaya Gorbatka,RU,2022-03-27 17:58:26,55.8703,41.7641,22.41,83,100,15.35
373,373,Chesma,RU,2022-03-27 18:00:55,53.8111,60.6533,22.87,93,86,12.73
392,392,Keuruu,FI,2022-03-27 18:01:01,62.2667,24.7000,23.81,94,100,12.88
402,402,Merrill,US,2022-03-27 18:01:04,45.1805,-89.6835,22.68,40,0,16.11


In [9]:
preferred_cities_df.count()

City_ID       21
City          21
Country       21
Date          21
Lat           21
Lng           21
Max Temp      21
Humidity      21
Cloudiness    21
Wind Speed    21
dtype: int64

In [10]:
# Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name
50,Kyakhta,RU,20.95,50.3508,106.4494,
54,Nome,US,20.16,64.5011,-165.4064,
62,Jamestown,US,24.53,42.0970,-79.2353,
180,Korsakovo,RU,22.55,53.2677,37.3569,
223,Sovetskiy,RU,23.70,61.3614,63.5842,
246,Zabaykalsk,RU,21.31,49.6333,117.3167,
290,Krasnaya Gorbatka,RU,22.41,55.8703,41.7641,
373,Chesma,RU,22.87,53.8111,60.6533,
392,Keuruu,FI,23.81,62.2667,24.7000,
402,Merrill,US,22.68,45.1805,-89.6835,


In [11]:
# Set the parameters to search for a hotel in Paris.
params = {
    "radius": 5000,
    "types": "lodging",
    "key": g_key,
    "location": "48.8566, 2.3522"}

# Iterate through the DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
# Grab the first hotel from the results and store the name.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [12]:
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# Store the DataFrame Row.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

In [13]:
hotel_info

['\n<dl>\n<dt>Hotel Name</dt><dd>Gostinitsa "Yuzhnaya"</dd>\n<dt>City</dt><dd>Kyakhta</dd>\n<dt>Country</dt><dd>RU</dd>\n<dt>Max Temp</dt><dd>20.95 °F</dd>\n</dl>\n',
 '\n<dl>\n<dt>Hotel Name</dt><dd>Dredge 7 Inn</dd>\n<dt>City</dt><dd>Nome</dd>\n<dt>Country</dt><dd>US</dd>\n<dt>Max Temp</dt><dd>20.16 °F</dd>\n</dl>\n',
 '\n<dl>\n<dt>Hotel Name</dt><dd>Hampton Inn & Suites Jamestown</dd>\n<dt>City</dt><dd>Jamestown</dd>\n<dt>Country</dt><dd>US</dd>\n<dt>Max Temp</dt><dd>24.53 °F</dd>\n</dl>\n',
 '\n<dl>\n<dt>Hotel Name</dt><dd></dd>\n<dt>City</dt><dd>Korsakovo</dd>\n<dt>Country</dt><dd>RU</dd>\n<dt>Max Temp</dt><dd>22.55 °F</dd>\n</dl>\n',
 '\n<dl>\n<dt>Hotel Name</dt><dd>Apartment on Kievskaya 29</dd>\n<dt>City</dt><dd>Sovetskiy</dd>\n<dt>Country</dt><dd>RU</dd>\n<dt>Max Temp</dt><dd>23.7 °F</dd>\n</dl>\n',
 '\n<dl>\n<dt>Hotel Name</dt><dd>Zheleznyy Mir</dd>\n<dt>City</dt><dd>Zabaykalsk</dd>\n<dt>Country</dt><dd>RU</dd>\n<dt>Max Temp</dt><dd>21.31 °F</dd>\n</dl>\n',
 '\n<dl>\n<dt>Hote

In [14]:
# Add a heatmap of temperature for the vacation spots and a pop-up marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)

heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)

marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))